# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [112]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [129]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [130]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Mathematical Optimization Model for Box Packing

**Parameters:**

*  `n`: Number of items
*  `m`: Maximum number of boxes available
*  `w_i`: Weight of item `i` (for `i = 1, ..., n`)
*  `C`: Capacity of each box

**Decision Variables:**

*  `x_ij`: Binary variable equal to 1 if item `i` is packed in box `j`, and 0 otherwise (for `i = 1, ..., n` and `j = 1, ..., m`)
*  `y_j`: Binary variable equal to 1 if box `j` is used, and 0 otherwise (for `j = 1, ..., m`)

**Objective Function:**
Minimize the number of boxes used:

```
Minimize  ∑_(j=1)^m y_j 
```

**Constraints:**

1. **Each item must be packed in exactly one box:**
   ```
   ∑_(j=1)^m x_ij = 1, for all i = 1, ..., n
   ```
2. **The total weight of items in a box cannot exceed its capacity:**
   ```
   ∑_(i=1)^n w_i * x_ij ≤ C * y_j,  for all j = 1, ..., m
   ```
3. **A box cannot be used unless an item is packed in it:**
   ```
   y_j ≥ x_ij, for all i = 1, ..., n and j = 1, ..., m
   ```

**Variable Domains:**
*  `x_ij ∈ {0, 1}` for all `i = 1, ..., n` and `j = 1, ..., m`
*  `y_j ∈ {0, 1}` for all `j = 1, ..., m`

This model represents a classic bin-packing problem, formulated as a mixed-integer linear programming problem (MILP). It aims to minimize the number of boxes used while ensuring each item is packed, respecting box capacity limits, and only using boxes that contain items.


## 3. Generate the pyomo code

In [131]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [132]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
n = 5  # Number of items
m = 3  # Maximum number of boxes
w = {1: 10, 2: 15, 3: 8, 4: 20, 5: 12}  # Weight of each item
C = 30  # Capacity of each box

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)  # Set of items
model.J = pyo.RangeSet(1, m)  # Set of boxes

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)  # 1 if item i is in box j
model.y = pyo.Var(model.J, within=pyo.Binary)  # 1 if box j is used

# Objective function: Minimize number of boxes used
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Each item must be packed in exactly one box
def constraint_item_packed(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.constraint_item_packed = pyo.Constraint(model.I, rule=constraint_item_packed)

# Box capacity constraint
def constraint_box_capacity(model, j):
    return sum(w[i] * model.x[i, j] for i in model.I) <= C * model.y[j]
model.constraint_box_capacity = pyo.Constraint(model.J, rule=constraint_box_capacity)

# Box usage constraint
def constraint_box_usage(model, i, j):
    return model.y[j] >= model.x[i, j]
model.constraint_box_usage = pyo.Constraint(model.I, model.J, rule=constraint_box_usage)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal number of boxes:", pyo.value(model.obj))

# Display packing results
for j in model.J:
    if pyo.value(model.y[j]) == 1:
        print(f"Box {j}:", [i for i in model.I if pyo.value(model.x[i, j]) == 1])
```

**Explanation:**

1. **Sample Data:** I have provided sample data for `n`, `m`, `w`, and `C`. You should replace these with your actual data.
2. **Model Initialization:** The code creates a concrete Pyomo model.
3. **Sets:** Defines sets for items (`I`) and boxes (`J`).
4. **Variables:** Defines binary variables `x` and `y` representing item-box assignments and box usage.
5. **Objective Function:** Defines the objective to minimize the total number of boxes used.
6. **Constraints:**
   - `constraint_item_packed`: Ensures each item is packed in exactly one box.
   - `constraint_box_capacity`: Enforces the box capacity limit.
   - `constraint_box_usage`: Ensures a box is marked as used if an item is assigned to it.
7. **Solver:** You need to have a solver installed (like GLPK, CBC, or others) and specify it using `pyo.SolverFactory`.
8. **Results:** The code prints the solver status, termination condition, optimal number of boxes used, and the packing arrangement.

Remember to install Pyomo and a suitable solver. You can then run this code to get the optimized packing solution for your specific problem instance. 


## 4. Input problem data and try running the generated code

In [133]:
import pyomo.environ as pyo
# items = range(1,25)
# boxes = range(1,25)
# weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item

# capacity = 9
# max_boxes = 15

# Sample data (replace with actual data)
n = 24  # Number of items
m = 15  # Maximum number of boxes
w = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
  # Weight of each item
C = 9  # Capacity of each box

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)  # Set of items
model.J = pyo.RangeSet(1, m)  # Set of boxes

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)  # 1 if item i is in box j
model.y = pyo.Var(model.J, within=pyo.Binary)  # 1 if box j is used

# Objective function: Minimize number of boxes used
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Each item must be packed in exactly one box
def constraint_item_packed(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.constraint_item_packed = pyo.Constraint(model.I, rule=constraint_item_packed)

# Box capacity constraint
def constraint_box_capacity(model, j):
    return sum(w[i] * model.x[i, j] for i in model.I) <= C * model.y[j]
model.constraint_box_capacity = pyo.Constraint(model.J, rule=constraint_box_capacity)

# Box usage constraint
def constraint_box_usage(model, i, j):
    return model.y[j] >= model.x[i, j]
model.constraint_box_usage = pyo.Constraint(model.I, model.J, rule=constraint_box_usage)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal number of boxes:", pyo.value(model.obj))

# Display packing results
for j in model.J:
    if pyo.value(model.y[j]) == 1:
        print(f"Box {j}:", [i for i in model.I if pyo.value(model.x[i, j]) == 1])

Status: ok
Termination condition: optimal
Optimal number of boxes: 13.0
Box 1: [23]
Box 2: [3, 21]
Box 3: [9, 15]
Box 4: [6, 19]
Box 5: [8, 13]
Box 6: [4, 16]
Box 7: [10, 17]
Box 8: [7, 14]
Box 9: [2, 22]
Box 10: [11, 18]
Box 11: [20]
Box 14: [1, 5, 12]
Box 15: [24]


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [134]:
print(response.text)

### Mathematical Optimization Model for Box Packing

**Parameters:**

*  `n`: Number of items
*  `m`: Maximum number of boxes available
*  `w_i`: Weight of item `i` (for `i = 1, ..., n`)
*  `C`: Capacity of each box

**Decision Variables:**

*  `x_ij`: Binary variable equal to 1 if item `i` is packed in box `j`, and 0 otherwise (for `i = 1, ..., n` and `j = 1, ..., m`)
*  `y_j`: Binary variable equal to 1 if box `j` is used, and 0 otherwise (for `j = 1, ..., m`)

**Objective Function:**
Minimize the number of boxes used:

```
Minimize  ∑_(j=1)^m y_j 
```

**Constraints:**

1. **Each item must be packed in exactly one box:**
   ```
   ∑_(j=1)^m x_ij = 1, for all i = 1, ..., n
   ```
2. **The total weight of items in a box cannot exceed its capacity:**
   ```
   ∑_(i=1)^n w_i * x_ij ≤ C * y_j,  for all j = 1, ..., m
   ```
3. **A box cannot be used unless an item is packed in it:**
   ```
   y_j ≥ x_ij, for all i = 1, ..., n and j = 1, ..., m
   ```

**Variable Domains:**
*  `x_ij ∈ {0, 

In [135]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
n = 5  # Number of items
m = 3  # Maximum number of boxes
w = {1: 10, 2: 15, 3: 8, 4: 20, 5: 12}  # Weight of each item
C = 30  # Capacity of each box

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)  # Set of items
model.J = pyo.RangeSet(1, m)  # Set of boxes

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)  # 1 if item i is in box j
model.y = pyo.Var(model.J, within=pyo.Binary)  # 1 if box j is used

# Objective function: Minimize number of boxes used
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Each item must be packed in exactly one box
def constraint_item_packed(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.constraint_item_packed = pyo.Constraint(model.I, rule=constraint_item_packed)

# Box capacity constraint
def constraint_box